In [2]:
search_for = 127
start_from = 0
threads = 10
thread = 5
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.002089262008666992
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 262209281
type: [1, 1, 1, 1, 127] 127
cases of this type: 260144641
100000 8.594179843952439
200000 14.925204283780374
300000 17.82231732776742
400000 19.22221151398768
500000 19.054700604272384
600000 19.751773123985632
700000 18.695348470180097
800000 18.729183739471612
900000 18.868014657046242
1000000 16.70803889109609
1100000 8.841554000165969
1200000 18.738091666344534
1300000 18.379564002547426
1400000 18.425205677964737
1500000 18.58179616248607
1600000 19.60962791594356
1700000 18.51246233332016
1800000 18.34639520280244
1900000 18.818322345040798
2000000 18.944501206851207
2100000 4.901074274325781
2200000 20.91943101371072
2300000 22.285909984684828
2400000 21.97129554555877
2500000 21.94959099116623
2600000 22.646647796062826
2700000 21.070160737799945
2800000 20.842747938862384
2900000 21.037299476998317
3000000 20.7

29400000 16.557503195605687
29500000 16.845678446104454
29600000 16.016298688385515
29700000 11.241939775608865
29800000 11.309623005197867
29900000 16.027213324771168
30000000 16.708370610430663
30100000 16.4188125082441
30200000 17.840260058396918
30300000 17.33981307137155
30400000 17.97977944498682
30500000 11.7697182275724
30600000 14.151839113932166
30700000 17.93222599815862
30800000 17.77219385492305
30900000 17.62966278578052
31000000 9.32995002627851
31100000 15.644152641652731
31200000 17.601502241978967
31300000 17.380371015960574
31400000 17.3251441497851
31500000 16.46930169535428
31600000 15.870407397764625
31700000 16.243665539351472
31800000 5.361901465887561
31900000 16.317307646561748
32000000 15.757812604525112
32100000 16.12225176621286
32200000 16.3709530152886
32300000 17.40996262557561
32400000 17.08714451242579
32500000 15.441080177715346
32600000 10.148014264149149
32700000 17.350251143270956
32800000 17.47164153573873
32900000 17.733477288977006
33000000 17.8

59100000 14.798934564198436
59200000 15.378489874796935
59300000 15.355696845377167
59400000 15.478545859195123
59500000 15.736723287791284
59600000 15.140707685075924
59700000 15.195235452842908
59800000 14.612856965343024
59900000 8.328387174348181
60000000 12.551596810315852
60100000 13.412649332182266
60200000 13.785995545449431
60300000 13.265591505215866
60400000 12.47885234894934
60500000 6.6683937466673235
60600000 13.443393690941862
60700000 13.36239332460089
60800000 13.055131278016411
60900000 13.271056394507118
61000000 8.075510536982033
61100000 13.483424993133386
61200000 14.99859099222022
61300000 14.693901792665832
61400000 15.660205526824441
61500000 16.07786827782589
61600000 15.424445926848325
61700000 14.697696676938993
61800000 14.9992388605912
61900000 15.001260678326963
62000000 6.2390758936302175
62100000 13.59755095525239
62200000 13.300203807781985
62300000 13.387899526398133
62400000 13.508908763842976
62500000 6.719108729131471
62600000 12.656341572373716
62

88800000 6.822275068560927
88900000 9.714864732552535
89000000 11.818671102953788
89100000 8.016015245435202
89200000 7.546636585027104
89300000 11.195849464696376
89400000 10.622604462469598
89500000 6.846196836579854
89600000 11.289606148308252
89700000 11.224492160697196
89800000 11.652531897359403
89900000 12.222597085394943
90000000 12.5941993221167
90100000 12.342688047410814
90200000 12.710391620584263
90300000 12.654414285453221
90400000 12.443120459859944
90500000 12.327226906477545
90600000 11.761246747810116
90700000 11.37511275864506
90800000 11.570956615687013
90900000 4.812421153639732
91000000 11.414986273382889
91100000 11.435615926397336
91200000 3.819887240395578
91300000 11.591985622159061
91400000 11.391848536085522
91500000 5.409393542756839
91600000 11.88955046487233
91700000 11.252722249970105
91800000 11.233399995461388
91900000 12.088498146018907
92000000 12.065134389474068
92100000 12.606121509632018
92200000 12.454295460248064
92300000 12.511816558407931
9240

118000000 8.541590749740317
118100000 9.208386811044715
118200000 9.27232199034137
118300000 9.370258661545902
118400000 9.34274886303482
118500000 9.362127627958255
118600000 9.62039543664377
118700000 9.753652412358932
118800000 10.353549499652985
118900000 10.619305371092914
119000000 9.361846787286927
119100000 9.395210665002699
119200000 9.317018854761566
119300000 9.188682793979812
119400000 9.215682429951633
119500000 9.358894942183904
119600000 8.854668178468343
119700000 9.378331105399612
119800000 3.089307337662628
119900000 1.6972365605743807
120000000 6.604440209026637
120100000 8.973871302486554
120200000 8.978722787854547
120300000 9.188189278090649
120400000 9.091504427685768
120500000 9.21652156470392
120600000 8.999126326097858
120700000 9.467760140767336
120800000 9.475868044729005
120900000 11.052161912778367
121000000 10.582652394101592
121100000 10.044373550434763
121200000 9.876121405526595
121300000 9.537550524299292
121400000 9.794143474905823
121500000 9.902484

146800000 7.40136032913589
146900000 7.663859802468342
147000000 7.817334992858727
147100000 7.77377520455112
147200000 7.7942817578792996
147300000 7.870989208702803
147400000 8.392092112460434
147500000 8.444568262936212
147600000 8.666556128968088
147700000 8.097398171407288
147800000 7.651163795314284
147900000 7.819323607907331
148000000 7.786117733533621
148100000 7.710452410421777
148200000 7.63534229080877
148300000 7.320164054642756
148400000 3.641227976034208
148500000 6.076440222795472
148600000 5.12547865041396
148700000 3.307131320896101
148800000 7.42786301678748
148900000 7.688815112453559
149000000 7.72210332727586
149100000 7.642532184698282
149200000 7.690340070007458
149300000 7.566511023756574
149400000 8.030424306084521
149500000 8.463602923621195
149600000 8.306853653716315
149700000 8.322386692407587
149800000 7.6601242309161695
149900000 7.5942973404420915
150000000 7.645884386905353
150100000 7.630414345322117
150200000 7.398403394223112
150300000 7.27737308204

176100000 6.285232094169906
176200000 6.364157701868859
176300000 6.279548207527371
176400000 6.308757962723237
176500000 6.131092557070802
176600000 6.022729424349758
176700000 5.66818063581971
176800000 4.647002511531439
176900000 3.7985123590079963
177000000 5.671939849249563
177100000 5.8372150015425905
177200000 3.0994215189439833
177300000 4.760143201475871
177400000 5.678528091761515
177500000 5.681173613025768
177600000 2.4437142390340147
177700000 5.744248542573553
177800000 5.629982730195064
177900000 6.1931707298209275
178000000 6.031214381401851
178100000 6.229235481528844
178200000 6.208703944797396
178300000 6.184467344730822
178400000 6.028437703718897
178500000 5.91245066226255
178600000 6.0277174306887735
178700000 5.62803678601858
178800000 5.5195821473145585
178900000 2.7184622143666393
179000000 5.568807622561194
179100000 5.571385278230157
179200000 5.349684753396411
179300000 2.2837314600756646
179400000 5.35696050704645
179500000 5.793092748079272
179600000 4.834

205300000 2.037911951093038
205400000 4.119911303621882
205500000 3.994404332873915
205600000 3.8785533998437893
205700000 3.8745479853924336
205800000 3.842419860909229
205900000 1.406195587874529
206000000 4.022950006782103
206100000 3.8746131862355884
206200000 3.816807033508523
206300000 3.8545099329204997
206400000 4.096749808683209
206500000 1.72269232110501
206600000 4.326731035519785
206700000 4.293273417026771
206800000 4.256846352208507
206900000 4.237370954687365
207000000 4.268976173105486
207100000 4.200205255511473
207200000 4.2183289770745755
207300000 2.875534200927012
207400000 3.0324770234693577
207500000 3.8818346585122887
207600000 3.818327262160398
207700000 3.5617490379728847
207800000 3.6038805541846033
207900000 2.842591875001046
208000000 2.302034276693184
208100000 3.604157348812743
208200000 3.602114089364252
208300000 3.77584371553516
208400000 3.8272645833233354
208500000 3.562253456895262
208600000 2.104608877679161
208700000 4.089262100330587
208800000 4.

234100000 2.1171486409141185
234200000 2.1409834325104655
234300000 2.030096327024925
234400000 1.9997247269378344
234500000 2.0107634102622867
234600000 0.6927543361229109
234700000 1.958128746627487
234800000 1.9849298941179454
234900000 2.050355190113483
235000000 2.0671228838586377
235100000 2.0332914274585505
235200000 2.109540842074564
235300000 2.0662466287408727
235400000 0.883189393570912
235500000 2.1110034547870806
235600000 2.099794838770572
235700000 2.058629994045041
235800000 0.9066696674790361
235900000 2.023572246638597
236000000 1.9855100870203692
236100000 1.928687207519196
236200000 1.8756617303926053
236300000 1.933930142580539
236400000 1.8513453411824847
236500000 1.776396518694483
236600000 1.0434267162718693
236700000 1.4753272247699931
236800000 1.7816875135221646
236900000 1.8277911755827387
237000000 1.9443899518156362
237100000 1.8715676607528609
237200000 1.8853261134400545
237300000 1.8957207128632698
237400000 1.7687153789599028
237500000 0.9822508472576

262100000 0.0007926384489178658
best so far: 0
type: [1, 1, 127, 1, 1] 127
cases of this type: 16129
262200000 5.794349973400434e-05
best so far: 0
type: [1, 127, 1, 1, 1] 127
cases of this type: 127
best so far: 0
type: [127, 1, 1, 1, 1] 127
cases of this type: 1
best so far: 0
all cases: 262209281
best: 0
